# STEP 2.1 - Create Ingredients Table

In [0]:
import pandas as pd
import re 
import ast
import re
from itertools import chain
from tqdm import notebook
from fractions import Fraction
tqdm.pandas()
import unicodedata

## Import recipe table
Specify the location of the recipe table

In [0]:
RECIPE_PATH = '/content/drive/My Drive/Data Science Projects/RecipeApp/data/myData/ALL_RECIPES_FINAL.csv'

In [0]:
R_DF = pd.read_csv(RECIPE_PATH, index_col=0)

In [0]:
R_DF = R_DF.drop_duplicates().reset_index().drop(columns=['index']).fillna('')

In [0]:
SEPARATOR_RE = re.compile(r'^([\d\s*[\d\.,/]*)\s*(.+)')


def normalize(st):
    return re.sub(r'\s+', ' ', SEPARATOR_RE.sub('\g<1> \g<2>', st)).strip()


def escape_re_string(text):
    text = text.replace('.', '\.')
    return re.sub(r'\s+', ' ', text)

UNITS = {"cup": ["cups", "cup", "c.", "c"], "fluid_ounce": ["fl. oz.", "fl oz", "fluid ounce", "fluid ounces"],
         "gallon": ["gal", "gal.", "gallon", "gallons"], "ounce": ["oz", "oz.", "ounce", "ounces"],
         "pint": ["pt", "pt.", "pint", "pints"], "pound": ["lb", "lb.", "pound", "pounds"],
         "quart": ["qt", "qt.", "qts", "qts.", "quart", "quarts"],
         "tablespoon": ["tbsp.", "tbsp", "T", "T.", "tablespoon", "tablespoons", "tbs.", "tbs"],
         "teaspoon": ["tsp.", "tsp", "t", "t.", "teaspoon", "teaspoons"],
         "gram": ["g", "g.", "gr", "gr.", "gram", "grams"], "kilogram": ["kg", "kg.", "kilogram", "kilograms"],
         "liter": ["l", "l.", "liter", "liters"], "milligram": ["mg", "mg.", "milligram", "milligrams"],
         "milliliter": ["ml", "ml.", "milliliter", "milliliters"], "pinch": ["pinch", "pinches"],
         "dash": ["dash", "dashes"], "touch": ["touch", "touches"], "handful": ["handful", "handfuls"],
         "stick": ["stick", "sticks"], "clove": ["cloves", "clove"], "can": ["cans", "can"], "large": ["large"],
         "small": ["small"], "scoop": ["scoop", "scoops"], "filets": ["filet", "filets"], "sprig": ["sprigs", "sprig"]}

NUMBERS = ['seventeen', 'eighteen', 'thirteen', 'nineteen', 'fourteen', 'sixteen', 'fifteen', 'seventy', 'twelve',
           'eleven', 'eighty', 'thirty', 'ninety', 'twenty', 'seven', 'fifty', 'sixty', 'forty', 'three', 'eight',
           'four', 'zero', 'five', 'nine', 'ten', 'one', 'six', 'two', 'an']

prepositions = ["of"]

a = list(chain.from_iterable(UNITS.values()))
a.sort(key=lambda x: len(x), reverse=True)
a = map(escape_re_string, a)
PARSER_RE = re.compile(
    r'(?P<quantity>(?:[\d\.,][\d\.,\s/]*)?\s*(?:(?:%s)\s*)*)?(\s*(?P<unit>%s)\s+)?(\s*(?:%s)\s+)?(\s*(?P<name>.+))?' % (
        '|'.join(NUMBERS), '|'.join(a), '|'.join(prepositions)))

## Parse all ingredients in the recipe table
Which will create a new table of ingredients which links to the index of the recipe to which it belongs. This will take a while

In [33]:
ingredients = []


for index, row in notebook.tqdm(R_DF.iterrows(), total=R_DF.shape[0]):
  try:
    for ingredient in ast.literal_eval(row['ingredients']):
        ingredients.append({'name': ingredient, 'recipe_index': index})
  except:
    print(row)
    # raise


title           
rating          
total_time      
yeilds          
ingredients     
instructions    
category        
Name: 424, dtype: object



In [34]:
ingr_df = pd.DataFrame(ingredients)
ingr_df

,name,recipe_index
0,"2 teaspoons canola oil, divided",0
1,2 cups diced apples,0
2,"2 tablespoons white sugar, divided",0
3,2 teaspoons ground cinnamon,0
4,¾ cup milk,0
...,...,...
640442,"3/4 cup milk, warmed",68633
640443,"2 large eggs, separated",68633
640444,"2 tablespoons unsalted butter, melted",68633
640445,"1 teaspoon vegetable oil, or as needed",68633


In [0]:
ingr_df = ingr_df['name'].str.extract(PARSER_RE, expand=False).drop(columns=[1,3,4])

In [36]:
ingr_df.head()

,quantity,unit,name
0,2,teaspoons,"canola oil, divided"
1,2,cups,diced apples
2,2,tablespoons,"white sugar, divided"
3,2,teaspoons,ground cinnamon
4,,NaN,¾ cup milk


## Parse fractions to decimal

In [0]:
def fraction_to_decimal(string):
    words = string.split()
#     print(words)
    sum = 0
    
    for word in words:
        try:
            try:
                sum = sum + float(Fraction(word))
            except:
                sum = sum + float(word)
        except:
            continue
    tqdm._instances.clear()
    return sum

In [0]:
def parse_vulgar_fraction(df):
    match = re.findall('[½⅓⅔¼¾⅕⅖⅗⅘⅙⅚⅐⅛⅜⅝⅞⅑⅒]', df['name'])
    if len(match) != 0:
        df['quantity'] =  float(df['quantity']) + float(unicodedata.numeric(match[0]))
        df['name'] = df['name'].replace(match[0], '').strip()
    return df

def parse_subunit(df):
    match = re.findall('\(.+\)', df['name'])
    if len(match) != 0:
        qty_match = re.findall('[\d./ ]+', match[0])
        unit_match = re.findall('[a-zA-Z]+', match[0])
        df['sub_qty'] = qty_match[0] if len(qty_match) != 0 else 0
        df['sub_unit'] = unit_match[0] if len(unit_match) != 0 else ''
        df['name'] = df['name'].replace(match[0], '').strip()
    return df

def parse_modifier(df):
    match = re.findall(',.*', df['name'])
    if len(match) != 0:
        df['modifier'] = match[0].strip(',').strip()
        df['name'] = df['name'].replace(match[0], '')
    return df

## Convert quantity to float
This function also converts fraction strings to float

In [41]:
ingr_df['quantity'] = ingr_df['quantity'].progress_apply(fraction_to_decimal)

100%|██████████| 640447/640447 [00:04<00:00, 144794.46it/s]


## Parse Subunits
For rows with a subunit (ex: '1 (8 ounce) can'), parse the sub quantity and unit

In [42]:
ingr_df = ingr_df.progress_apply(parse_subunit, axis=1)

100%|██████████| 640447/640447 [05:40<00:00, 1881.99it/s]


## Convert Sub Quantity to Float
Convert the rows with a sub qty with a fraction to float

In [43]:
ingr_df['sub_qty'] = ingr_df['sub_qty'].astype(str).progress_apply(fraction_to_decimal)

100%|██████████| 640447/640447 [00:03<00:00, 170692.25it/s]


## Parse modifier
To clean up the ingredient, parse the modifier (i.e. anything after a comma such as: '1 cup cheese, shredded') and put into separate column

In [44]:
ingr_df = ingr_df.progress_apply(parse_modifier, axis=1)

100%|██████████| 640447/640447 [05:46<00:00, 1847.44it/s]


## Parse Vulgar Fractions
Convert vulgar fractions (i.e. '½') to float

In [45]:
ingr_df = ingr_df.progress_apply(parse_vulgar_fraction, axis=1)

100%|██████████| 640447/640447 [01:42<00:00, 6231.97it/s]


## Parse Unit Again
For rows with vulgar fractions, the unit was not parsed initially

In [0]:
test = ingr_df['name'].str.extract(PARSER_RE, expand=False).drop(columns=[1,3,4])

In [47]:
test = test[test.unit.isna() == False]
for index, row in notebook.tqdm(test.iterrows(), total=test.shape[0]):
    ingr_df.at[index, 'unit'] = row['unit']
    ingr_df.at[index, 'name'] = row['name']

100%|██████████| 93063/93063 [00:13<00:00, 7151.69it/s]


In [48]:
def fix_chicken_breast(df):
    if 'skinless' in df['name']:
        df['name'] = 'chicken breast'
    return df
        
ingr_df = ingr_df.progress_apply(fix_chicken_breast, axis=1)

100%|██████████| 640447/640447 [01:33<00:00, 6816.86it/s]


## Save ingredients to csv

In [0]:
ingr_df['recipe_index'] = pd.DataFrame(ingredients)['recipe_index']

In [0]:
ingr_df.to_csv('/content/drive/My Drive/Data Science Projects/RecipeApp/data/myData/INGREDIENTS_FINAL.csv')